In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver, get_xpath
from method.ours.parse import parse_form
from method.ours.prompts import get_form_context
from method.ours.history import HistoryTable
from method.ours.constraints import Invalid
from method.ours.generation import (
    generate_constraints_for_input_groups,
    generate_value_for_input_group,
    generate_values_for_input_groups,
    fill_form_with_value_table,
    submit_form
)
from method.ours.feedback import (
    get_local_feedback,
    get_global_feedback
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

URL = 'http://localhost:9000/dashboard/customers/add'
# 'https://www.mbta.com/'
# 'https://www.united.com/en/us'
# 'https://www.uhaul.com/Truck-Rentals/'
# 'http://localhost:8080/1/common/items/create'
# 'https://www.cars.com/'
# 'https://www.ups.com/ca/en/Home.page'
# 'https://www.healthgrades.com/find-a-doctor'
# 'https://www.babycenter.com/pregnancy-weight-gain-estimator'
# 'https://www.budget.com/en/home'
# 'https://www.yelp.com/'
# 'https://www.aircanada.com/ca/en/aco/home.html'

In [5]:
def get_to_form(driver):
    try:
        driver.get(URL)
    except:
        print('timeout')
    
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[2]/DIV[1]/INPUT[1]')
        )
    ).send_keys('admin@example.com')
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[4]/DIV[1]/DIV[1]/INPUT[1]')
        )
    ).send_keys('admin')
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[5]/BUTTON[1]')
        )
    ).click()

    '''
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]')
        )
    ).click()
    '''
    
    '''
    try:
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.ID,
                'email')
            )
        ).send_keys('me@company.com')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.ID,
                'password')
            )
        ).send_keys('password')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//*[@type="submit"]')
            )
        ).click()
    except:
        pass
    '''
    
    '''
    # Pet Clinic - Add Owner
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]')
        )
    ).click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/UL[1]/LI[2]')
        )
    ).click()
    '''
    
    '''
    # UPS - Quote
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'tabs_0_tab_1'))
    ).click()
    '''
    
    '''
    # AC - Multi-city
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
    ).click()
    '''
    
    '''
    # AC - My Bookings
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmg-tab-button-mngBook'))
    ).click()'''
    
    '''
    # Budget - Reservation
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'PicLoc_value'))
    ).click()
    '''

In [9]:
def find_form():
    return WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.TAG_NAME,
            'form'
        ))
    )


def find_button():
    '''return find_form().find_element(
        By.XPATH,
        '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[2]/MAIN[1]/FORM[1]'
    )'''
    # return None
    return driver.find_elements(By.TAG_NAME, 'button')[7]


In [7]:
driver = create_driver(HEADLESS)
get_to_form(driver)

[WDM] - Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 8.29M/8.29M [00:00<00:00, 17.0MB/s]


In [27]:
form = find_form()

In [11]:
history_table = HistoryTable(
    url=URL,
    xpath=get_xpath(driver, form)
)

In [12]:
for element in form.find_elements(By.TAG_NAME, 'input'):
    try:
        element.clear()
    except:
        pass

# Processing

In [28]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
relation_graph, input_groups = parse_form(
    driver,
    form,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/147 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:06<00:00,  5.33it/s]


# Generation

In [ ]:
value_table = generate_constraints_for_input_groups(input_groups)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

In [15]:
value_table = generate_values_for_input_groups(input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Johnson",
        "role": "assistant"
      }
    }
  ],
  "created": 1690307261,
  "id": "chatcmpl-7gGQH3K9613Jyt8ieHmKc3H8EgrrM",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 287,
    "total_tokens": 288
  }
}
  Total running cost: $0.309
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Los Angeles",
        "role": "assistant"
      }
    }
  ],
  "created": 1690307266,
  "id": "chatcmpl-7gGQMzKUVUiceAUj4we50CesMAjr0",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 2,
    "prompt_tokens": 258,
    "total_tokens": 260
  }
}
  Total running cost: $0.350
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in ques

In [16]:
fill_form_with_value_table(driver, value_table, input_groups)

<input></input> at y: (140, 188), x: (59, 501)
Alexander
<input></input> at y: (140, 188), x: (514, 956)
Anderson
<input></input> at y: (208, 256), x: (59, 501)
primaryaddress@gmail.com
<input></input> at y: (395, 443), x: (59, 501)
Alexander
<input></input> at y: (395, 443), x: (514, 956)
Johnson
<input></input> at y: (463, 511), x: (59, 501)
Microsoft
<input></input> at y: (463, 511), x: (514, 956)
1234567890
<input></input> at y: (531, 579), x: (59, 955)
123 Main Street
<input></input> at y: (599, 647), x: (59, 955)
123 Main Street Apartment 4B
<input></input> at y: (667, 715), x: (59, 501)
Los Angeles
<input></input> at y: (667, 715), x: (514, 956)
123456789
<input></input> at y: (735, 783), x: (59, 467)
Meeting scheduled for next week to discuss project progress.
<textarea></textarea> at y: (922, 942), x: (59, 927)
Thank you for your feedback.
unable to fill input //BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[2]/MAIN[1]/FORM[1]/DIV[1]/DIV[2]/DIV[1]/DIV[5]/DIV[2]/DIV[3]/DIV[1]/TEXTAREA[2] Messa

In [15]:
submit_form(driver, input_groups=input_groups, explicit_submit=find_button())

In [16]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

In [17]:
history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Feedback

In [27]:
new_form = find_form()

In [28]:
new_relation_graph, new_input_groups = parse_form(
    driver,
    new_form,
    prev_relation_graph=relation_graph,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/87 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:08<00:00,  3.78it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [29]:
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

In [30]:
global_feedback

[]

In [31]:
value_table = generate_constraints_for_input_groups(
    new_input_groups,
    value_table,
    global_feedback
)

In [32]:
value_table = generate_values_for_input_groups(new_input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "12",
        "role": "assistant"
      }
    }
  ],
  "created": 1689963115,
  "id": "chatcmpl-7eotXadZLKoGegZ5kGQIPykZt1jzX",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 295,
    "total_tokens": 296
  }
}
  Total running cost: $0.369
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and

In [33]:
fill_form_with_value_table(driver, value_table, new_input_groups)

<input></input> at y: (464, 518), x: (185, 752)
123-456 Main Street, Los Angeles
<input></input> at y: (464, 518), x: (784, 1351)
90210
<input>value is true</input> at y: (535, 548), x: (783, 796)
true
<input></input> at y: (628, 682), x: (185, 353)
5.2
<input></input> at y: (628, 682), x: (385, 553)
12
<input></input> at y: (628, 682), x: (584, 752)
15
<input></input> at y: (628, 682), x: (784, 952)
12


In [34]:
submit_form(driver, input_groups=new_input_groups, explicit_submit=find_button)

In [35]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Constraint Negation

In [18]:
form_context = get_form_context(input_groups)

In [19]:
# history_table.table = history_table.table[:1]

In [20]:
get_attr = lambda x, a, d: x[a] if a in x else d

for field_id, entry in value_table.entries.items():
    input_type = get_attr(entry.input_group.node.element.attrs, 'type', 'text')
    if input_type in ['radio', 'checkbox']:
        continue
    
    for i in range(len(entry.constraints) + 1):
        get_to_form(driver)
        
        value_table_copy = value_table.copy()
    
        if i == 0:
            value_table_copy.entries[field_id].constraints = [Invalid()]
        else:
            value_table_copy.entries[field_id].constraints = [value_table.entries[field_id].constraints[i - 1].copy()]
            value_table_copy.entries[field_id].constraints[0].flip_negative()


        input_group = value_table_copy.entries[field_id].input_group

        value_table_copy = generate_value_for_input_group(
            input_group,
            value_table_copy,
            form_context=form_context
        )

        fill_form_with_value_table(driver, value_table_copy, input_groups)

        try:
            driver.find_element(By.TAG_NAME, 'body').click()
            submit_form(driver, input_groups=input_groups, explicit_submit=find_button())
        except:
            pass
        
        time.sleep(1)
        
        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html)
        
        print(global_feedback)
        
        history_table.add(
            value_table_copy.get_values_dict(),
            str(type(value_table_copy.entries[field_id].constraints[0])),
            global_feedback,
            driver.current_url
        )
        
        if len(global_feedback) == 0:
            value_table.entries[field_id].constraints[i - 1].approve()

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


In [21]:
driver.get(URL)
time.sleep(1)
base_text = driver.find_element(By.TAG_NAME, 'body').text

In [21]:
for i, v in enumerate(history_table.table['values']):
    jv = json.loads(v)
    for key, value in jv.items():
        if key not in history_table.table.columns:
            history_table.table[key] = None
        history_table.table.loc[i, key] = value

In [22]:
history_table.table

,values,variation_type,feedback,new_url,//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/DIV[2]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1],//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/DIV[2]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[3]/DIV[1]/INPUT[1]
0,"{""//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/D...",base,Trip Planner\n\n\n\n\nNo trips available\n ...,https://www.mbta.com/trip-planner?_utf8=%E2%9C...,Boston,New York
1,"{""//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/D...",<class 'method.ours.constraints.concrete.Inval...,,https://www.mbta.com/,Mars,New York
2,"{""//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/D...",<class 'method.ours.constraints.concrete.ToBeT...,,https://www.mbta.com/,,New York
3,"{""//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/D...",<class 'method.ours.constraints.concrete.Inval...,,https://www.mbta.com/,Boston,Mars
4,"{""//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/D...",<class 'method.ours.constraints.concrete.ToBeT...,,https://www.mbta.com/,Boston,
5,"{""//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/D...",<class 'method.ours.constraints.concrete.ToBe'>,,https://www.mbta.com/,Boston,Boston


In [49]:
history_table.table = history_table.table[
    history_table.table.feedback.str.len() < len(base_text)
]

In [31]:
set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER'))

{"Close\nSelect a Nearby Location\n\n\n\n\n\n\n\n\n\n0\n\t\t\t\t\t\t\tCar Rental Locations Found\n\n\n\nThere are no locations for this city. Please enter a new search or click here to browse for a location.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPlease clarify your Budget location\n\n\nWe Can't find the location. Please select from matches below or enter a new search.\n\nPossible Matches",
 "Close\nSelect a Nearby Location\n\n\n\n\n\n\n\n\n\n43\n\t\t\t\t\t\t\tCar Rental Locations Found\n\n\n\n\n\n\n\n\n1\nUnion Station (Amtrak)\n\n\n\n0.5 miles away\n\nAddress: 800 N Alameda St, (Near Cesar Chavez Ave), Los Angeles, US\n\n\nPhone:\n\t\t\t\t\t\t\t\t\t\t(1) 213-617-2977\n\t\t\t\t\t\t\t\t\t\n\nLocation Type:\nCorporate\n\n\n\nHours of Operation: Mon - Fri 8:00 AM - 5:00 PM; Sat 8:00 AM - 1:00 PM\n\t\t\t\t\t\t\t\t\t\n\nSelect This Location\n\n\n\n\n\n\n2\nThe Bloc Shopping Center\n\n\n\n0.9 miles away\n\nAddress: 700 W 7th St - Ste A200, (Inside The Bloc Garage), Los Angeles, US\n\n\nPhone:\n

In [34]:
len(set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER')))

7

In [23]:
history_table.to_test_case('aoc-multitrip.py')